In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [5]:
submissson = pd.read_csv('submission.csv')

In [8]:
from preprocess import preprocess

In [9]:
test, colnames = preprocess("test.csv")

In [10]:
test.head()

,ego_id,u,v,t,x1,x2,x3,city_dist,university_dist,sex,...,gen_1,sex2_0,sex2_1,sex2_2,city_0,city_1,city_2,university_0,university_1,university_2
0,8,20,19,185.7,3.839089e-04,0.0,0.0,0,284912378,0,...,True,False,False,True,False,False,True,True,False,False
1,8,10,19,158.3,1.046045e+00,0.0,0.0,0,-1,0,...,False,False,False,True,False,False,True,True,False,False
2,8,2,19,594.5,7.529229e-01,0.0,0.0,0,188425734,0,...,True,False,False,True,False,False,True,True,False,False
3,8,8,19,NaN,1.751368e-06,0.0,0.0,-1,-1,1,...,False,True,False,False,True,False,False,True,False,False
4,8,20,2,164.1,3.034230e-13,0.0,0.0,0,473338112,0,...,True,False,False,True,False,False,True,True,False,False


In [11]:
colnames

Index(['x2', 'x3', 'city_dist', 'university_dist', 'sex', 'school_dist',
       'age_dist', 'age_mean', 'school', 'sum', 'u_v', 'gen_0', 'gen_1',
       'sex2_0', 'sex2_1', 'sex2_2', 'city_0', 'city_1', 'city_2',
       'university_0', 'university_1', 'university_2'],
      dtype='object')

In [12]:
import pickle


In [13]:
# Загрузка модели
with open ('model_dum.pkl', 'rb') as f:
    myModel = pickle.load(f)

In [22]:
colnames1 = colnames.drop('sum')

In [66]:
from config import *
from modelerWG import predict, rmse

preds = predict(test, myModel, colnames=colnames1)

In [29]:
test["preds"] = preds

In [43]:
# Соединение Submission и Test
res = submissson.merge(test, left_on=["ego_id", "u", "v"], right_on=["ego_id", "u", "v"], how="left")

In [45]:
# Вывод только нужного
res = res[["u","v", "preds", "x1_x"]]

,u,v,preds,x1_x
0,0,93,0.373477,0.000000
1,0,143,0.373477,0.000000
2,0,151,1.888285,1.606742
3,1,24,0.476123,0.026496
4,5,4,0.623399,0.159857
...,...,...,...,...
811014,73,23,0.623399,1.562170
811015,74,68,0.748413,0.454210
811016,77,28,0.993018,0.078504
811017,79,38,0.476123,0.981812
